# Week 8 Homework SOLUTION --- Volatility Forecasting Showdown

## SOLUTION --- do not distribute

**Quantitative Finance ML Course**

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LinearRegression
from arch import arch_model
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)

np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# --- Generate synthetic data for 30 stocks ---
np.random.seed(42)
n_days = 2520
n_stocks = 30
tickers = [f'STOCK_{i:02d}' for i in range(n_stocks)]

all_data = {}
for ticker in tickers:
    omega = np.random.uniform(5e-7, 5e-6)
    alpha = np.random.uniform(0.04, 0.12)
    beta = np.random.uniform(0.82, 0.94)
    if alpha + beta >= 0.999:
        beta = 0.999 - alpha

    returns = np.zeros(n_days)
    sigma2 = np.zeros(n_days)
    sigma2[0] = omega / (1 - alpha - beta)

    for t in range(1, n_days):
        sigma2[t] = omega + alpha * returns[t-1]**2 + beta * sigma2[t-1]
        returns[t] = np.sqrt(sigma2[t]) * np.random.randn()

    close = 100 * np.exp(np.cumsum(returns))
    dv = np.sqrt(sigma2)
    high = close * np.exp(np.abs(np.random.randn(n_days)) * dv * 0.5)
    low = close * np.exp(-np.abs(np.random.randn(n_days)) * dv * 0.5)
    open_p = close * np.exp(np.random.randn(n_days) * dv * 0.2)
    high = np.maximum(high, np.maximum(close, open_p)) * 1.001
    low = np.minimum(low, np.minimum(close, open_p)) * 0.999

    all_data[ticker] = pd.DataFrame({
        'open': open_p, 'high': high, 'low': low, 'close': close,
        'return': returns,
        'volume': np.random.lognormal(18, 0.5, n_days)
    }, index=pd.bdate_range('2015-01-02', periods=n_days))

print(f'Generated {n_stocks} stocks, {n_days} days each')

---

## Part 1: Compute RV for 30 Stocks (15 pts) --- SOLUTION

In [ ]:
def rv_close_to_close(returns, window=5):
    """Close-to-close realized volatility."""
    return returns.pow(2).rolling(window).sum().apply(np.sqrt) * np.sqrt(252 / window)


def rv_parkinson(high, low, window=5):
    """Parkinson range-based RV."""
    log_hl = (np.log(high) - np.log(low)) ** 2
    return (log_hl / (4 * np.log(2))).rolling(window).mean().apply(np.sqrt) * np.sqrt(252)


def rv_garman_klass(open_p, high, low, close, window=5):
    """Garman-Klass RV."""
    log_hl = (np.log(high) - np.log(low)) ** 2
    log_co = (np.log(close) - np.log(open_p)) ** 2
    gk = 0.5 * log_hl - (2 * np.log(2) - 1) * log_co
    # Clamp negative values to small positive before sqrt
    gk_mean = gk.rolling(window).mean().clip(lower=1e-12)
    return gk_mean.apply(np.sqrt) * np.sqrt(252)


# Apply to all stocks
for ticker, stock_df in all_data.items():
    stock_df['rv_cc'] = rv_close_to_close(stock_df['return'])
    stock_df['rv_park'] = rv_parkinson(stock_df['high'], stock_df['low'])
    stock_df['rv_gk'] = rv_garman_klass(stock_df['open'], stock_df['high'],
                                         stock_df['low'], stock_df['close'])
    stock_df['rv_target'] = stock_df['rv_cc'].shift(-1)  # next-day target
    stock_df['log_volume'] = np.log(stock_df['volume'])
    stock_df.dropna(inplace=True)

print(f'RV computed for {n_stocks} stocks')
print(f'Sample stock has {len(all_data[tickers[0]])} valid rows')

In [ ]:
# --- Summary statistics ---
stats_list = []
for ticker in tickers[:5]:  # show first 5
    d = all_data[ticker]
    stats_list.append({
        'Ticker': ticker,
        'Mean RV(CC)': d['rv_cc'].mean(),
        'Std RV(CC)': d['rv_cc'].std(),
        'AC(1) RV(CC)': d['rv_cc'].autocorr(1),
        'Mean RV(Park)': d['rv_park'].mean(),
        'Mean RV(GK)': d['rv_gk'].mean()
    })

stats_df = pd.DataFrame(stats_list).set_index('Ticker')
print(stats_df.round(4).to_string())

In [ ]:
# --- Plot RV for 3 representative stocks ---
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

for i, ticker in enumerate([tickers[0], tickers[10], tickers[20]]):
    d = all_data[ticker]
    axes[i].plot(d.index, d['rv_cc'], label='Close-to-Close', alpha=0.8)
    axes[i].plot(d.index, d['rv_park'], label='Parkinson', alpha=0.8)
    axes[i].plot(d.index, d['rv_gk'], label='Garman-Klass', alpha=0.8)
    axes[i].set_ylabel('Ann. RV')
    axes[i].set_title(f'{ticker}')
    axes[i].legend(loc='upper right')

plt.xlabel('Date')
plt.suptitle('Realized Volatility Estimators', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

---

## Part 2: GARCH + HAR Baselines (20 pts) --- SOLUTION

In [ ]:
# --- GARCH(1,1) for all 30 stocks ---

garch_results = {}  # ticker -> test predictions

for ticker in tickers:
    d = all_data[ticker]
    n = len(d)
    train_end = int(n * 0.6)
    val_end = int(n * 0.8)

    # Fit on train+val, forecast on test
    rets_scaled = d['return'].values * 100
    am = arch_model(rets_scaled[:val_end], vol='GARCH', p=1, q=1, mean='Constant')
    res = am.fit(disp='off')

    # Simple approach: use fitted model to get conditional variances for test
    # Re-fit on all data up to val_end, then forecast
    forecasts = []
    for t in range(val_end, n):
        # Expanding window (keep model params fixed for speed)
        am_full = arch_model(rets_scaled[:t], vol='GARCH', p=1, q=1, mean='Constant')
        res_full = am_full.fit(disp='off', last_obs=t)
        fc = res_full.forecast(horizon=1)
        garch_vol = np.sqrt(fc.variance.values[-1, 0]) / 100 * np.sqrt(252)
        forecasts.append(garch_vol)

    garch_results[ticker] = {
        'pred': np.array(forecasts),
        'actual': d['rv_target'].values[val_end:val_end + len(forecasts)]
    }

print(f'GARCH fitted for {len(garch_results)} stocks')

In [ ]:
# --- HAR for all 30 stocks ---

har_results = {}

for ticker in tickers:
    d = all_data[ticker].copy()
    d['rv_d'] = d['rv_cc']
    d['rv_w'] = d['rv_cc'].rolling(5).mean()
    d['rv_m'] = d['rv_cc'].rolling(22).mean()
    d = d.dropna()

    n = len(d)
    train_end = int(n * 0.6)
    val_end = int(n * 0.8)

    har_feats = ['rv_d', 'rv_w', 'rv_m']
    train = d.iloc[:train_end]
    test = d.iloc[val_end:]

    model = LinearRegression()
    model.fit(train[har_feats], train['rv_target'])

    pred = model.predict(test[har_feats])

    har_results[ticker] = {
        'pred': pred,
        'actual': test['rv_target'].values,
        'coefs': dict(zip(har_feats, model.coef_))
    }

print(f'HAR fitted for {len(har_results)} stocks')
print(f'\nSample HAR coefficients ({tickers[0]}):')
print(har_results[tickers[0]]['coefs'])

---

## Part 3: LSTM Vol Forecaster (30 pts) --- SOLUTION

In [ ]:
class VolSequenceDataset(Dataset):
    """Sequence dataset for volatility forecasting."""
    def __init__(self, features, targets, seq_len=20):
        self.features = features.astype(np.float32)
        self.targets = targets.astype(np.float32)
        self.seq_len = seq_len

    def __len__(self):
        return len(self.features) - self.seq_len

    def __getitem__(self, idx):
        X = torch.from_numpy(self.features[idx:idx + self.seq_len])
        y = torch.tensor(self.targets[idx + self.seq_len])
        return X, y


class LSTMVolForecaster(nn.Module):
    """LSTM for volatility forecasting."""
    def __init__(self, input_dim, hidden_dim=32, n_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0.0
        )
        self.head = nn.Sequential(
            nn.Linear(hidden_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_hidden = lstm_out[:, -1, :]
        return self.head(last_hidden).squeeze(-1)


class EarlyStopping:
    def __init__(self, patience=15):
        self.patience = patience
        self.best_loss = float('inf')
        self.counter = 0
        self.best_state = None

    def step(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            self.best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            return False
        self.counter += 1
        return self.counter >= self.patience

    def restore_best(self, model):
        if self.best_state:
            model.load_state_dict(self.best_state)


print('Classes defined.')

In [ ]:
# --- Train LSTM for each stock ---

SEQ_LEN = 20
lstm_features_cols = ['rv_cc', 'return', 'log_volume']
lstm_results = {}

for idx, ticker in enumerate(tickers):
    d = all_data[ticker].copy()
    n = len(d)
    train_end = int(n * 0.6)
    val_end = int(n * 0.8)

    # Extract arrays
    feats = d[lstm_features_cols].values
    targets = d['rv_target'].values

    # Normalize using train stats
    train_feats = feats[:train_end]
    feat_mean = train_feats.mean(axis=0)
    feat_std = train_feats.std(axis=0) + 1e-8
    feats_norm = (feats - feat_mean) / feat_std

    # Create datasets (with overlap for sequence start)
    train_ds = VolSequenceDataset(feats_norm[:train_end], targets[:train_end], SEQ_LEN)
    val_ds = VolSequenceDataset(feats_norm[train_end - SEQ_LEN:val_end],
                                 targets[train_end - SEQ_LEN:val_end], SEQ_LEN)
    test_ds = VolSequenceDataset(feats_norm[val_end - SEQ_LEN:],
                                  targets[val_end - SEQ_LEN:], SEQ_LEN)

    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=256, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=256, shuffle=False)

    # Train
    torch.manual_seed(42)
    model = LSTMVolForecaster(input_dim=len(lstm_features_cols))
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    stopper = EarlyStopping(patience=15)

    for epoch in range(150):
        model.train()
        for X_b, y_b in train_loader:
            optimizer.zero_grad()
            loss = ((model(X_b) - y_b) ** 2).mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        model.eval()
        v_losses = []
        with torch.no_grad():
            for X_b, y_b in val_loader:
                v_losses.append(((model(X_b) - y_b) ** 2).mean().item())
        if stopper.step(np.mean(v_losses), model):
            break

    stopper.restore_best(model)

    # Predict on test
    model.eval()
    preds, actuals = [], []
    with torch.no_grad():
        for X_b, y_b in test_loader:
            preds.append(model(X_b).numpy())
            actuals.append(y_b.numpy())

    lstm_results[ticker] = {
        'pred': np.concatenate(preds),
        'actual': np.concatenate(actuals)
    }

    if (idx + 1) % 10 == 0:
        print(f'  {idx+1}/{n_stocks} stocks done')

print(f'\nLSTM trained for {len(lstm_results)} stocks')

---

## Part 4: Compare All Models (20 pts) --- SOLUTION

In [ ]:
def compute_mse(actual, pred):
    return np.mean((actual - pred) ** 2)


def compute_qlike(actual, pred):
    """QLIKE loss for volatility."""
    var_true = actual ** 2 + 1e-10
    var_pred = pred ** 2 + 1e-10
    ratio = var_true / var_pred
    return np.mean(ratio - np.log(ratio) - 1)


def compute_corr(actual, pred):
    return np.corrcoef(actual, pred)[0, 1]


# Compute metrics for each stock x model
per_stock_metrics = []

for ticker in tickers:
    # Align lengths
    n_test = min(
        len(garch_results[ticker]['pred']),
        len(har_results[ticker]['pred']),
        len(lstm_results[ticker]['pred'])
    )

    actual = har_results[ticker]['actual'][:n_test]  # common actual
    garch_p = garch_results[ticker]['pred'][:n_test]
    har_p = har_results[ticker]['pred'][:n_test]
    lstm_p = lstm_results[ticker]['pred'][:n_test]

    for model_name, pred in [('GARCH', garch_p), ('HAR', har_p), ('LSTM', lstm_p)]:
        per_stock_metrics.append({
            'Ticker': ticker,
            'Model': model_name,
            'MSE': compute_mse(actual, pred),
            'QLIKE': compute_qlike(actual, pred),
            'Correlation': compute_corr(actual, pred)
        })

metrics_df = pd.DataFrame(per_stock_metrics)

# Aggregate across stocks
agg_metrics = metrics_df.groupby('Model')[['MSE', 'QLIKE', 'Correlation']].mean()
print('Average Metrics Across 30 Stocks:')
print(agg_metrics.round(6).to_string())

In [ ]:
# --- Comparison bar chart ---
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, metric in enumerate(['MSE', 'QLIKE', 'Correlation']):
    vals = agg_metrics[metric]
    colors = ['#e74c3c', '#3498db', '#2ecc71']
    axes[i].bar(vals.index, vals.values, color=colors)
    axes[i].set_title(metric)
    axes[i].set_ylabel(metric)
    for j, v in enumerate(vals.values):
        axes[i].text(j, v, f'{v:.4f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('Model Comparison (Average Across 30 Stocks)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# --- Per-stock analysis: where does LSTM win? ---

# For each stock, find the best model by QLIKE
best_model_per_stock = metrics_df.loc[
    metrics_df.groupby('Ticker')['QLIKE'].idxmin()
][['Ticker', 'Model', 'QLIKE']]

win_counts = best_model_per_stock['Model'].value_counts()
print('Best Model by QLIKE (per stock):')
print(win_counts)
print(f'\nLSTM wins for {win_counts.get("LSTM", 0)}/{n_stocks} stocks')
print(f'HAR wins for {win_counts.get("HAR", 0)}/{n_stocks} stocks')
print(f'GARCH wins for {win_counts.get("GARCH", 0)}/{n_stocks} stocks')

In [ ]:
# --- Full comparison table (top 10 stocks) ---
pivot = metrics_df.pivot_table(
    index='Ticker', columns='Model', values=['MSE', 'QLIKE', 'Correlation']
)
print('Per-Stock Metrics (first 10 stocks):')
print(pivot.head(10).round(4).to_string())

---

## Part 5: Attention Extension (15 pts) --- SOLUTION

In [ ]:
class LSTMWithAttention(nn.Module):
    """LSTM with attention over hidden states."""
    def __init__(self, input_dim, hidden_dim=32, n_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0.0
        )
        # Attention: project hidden states, then dot with learned query
        self.attn_proj = nn.Linear(hidden_dim, hidden_dim)
        self.attn_query = nn.Parameter(torch.randn(hidden_dim))

        self.head = nn.Sequential(
            nn.Linear(hidden_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        # x: (batch, seq_len, input_dim)
        lstm_out, _ = self.lstm(x)  # (batch, seq_len, hidden_dim)

        # Attention scores
        proj = torch.tanh(self.attn_proj(lstm_out))  # (batch, seq_len, hidden_dim)
        scores = (proj * self.attn_query).sum(dim=-1)  # (batch, seq_len)
        weights = torch.softmax(scores, dim=-1)  # (batch, seq_len)

        # Weighted sum of hidden states
        context = (lstm_out * weights.unsqueeze(-1)).sum(dim=1)  # (batch, hidden_dim)

        return self.head(context).squeeze(-1)

    def get_attention_weights(self, x):
        """Return attention weights for visualization."""
        self.eval()
        with torch.no_grad():
            lstm_out, _ = self.lstm(x)
            proj = torch.tanh(self.attn_proj(lstm_out))
            scores = (proj * self.attn_query).sum(dim=-1)
            return torch.softmax(scores, dim=-1)


# Test
attn_model = LSTMWithAttention(input_dim=3)
x_test = torch.randn(8, 20, 3)
print(f'Output shape: {attn_model(x_test).shape}')
print(f'Attention weights shape: {attn_model.get_attention_weights(x_test).shape}')

In [ ]:
# --- Train attention model for a subset of stocks ---
# (For demonstration, train on 10 stocks to save time)

attn_results = {}
sample_tickers = tickers[:10]

for idx, ticker in enumerate(sample_tickers):
    d = all_data[ticker].copy()
    n = len(d)
    train_end = int(n * 0.6)
    val_end = int(n * 0.8)

    feats = d[lstm_features_cols].values
    targets = d['rv_target'].values

    train_feats = feats[:train_end]
    feat_mean = train_feats.mean(axis=0)
    feat_std = train_feats.std(axis=0) + 1e-8
    feats_norm = (feats - feat_mean) / feat_std

    train_ds = VolSequenceDataset(feats_norm[:train_end], targets[:train_end], SEQ_LEN)
    val_ds = VolSequenceDataset(feats_norm[train_end - SEQ_LEN:val_end],
                                 targets[train_end - SEQ_LEN:val_end], SEQ_LEN)
    test_ds = VolSequenceDataset(feats_norm[val_end - SEQ_LEN:],
                                  targets[val_end - SEQ_LEN:], SEQ_LEN)

    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=256, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=256, shuffle=False)

    torch.manual_seed(42)
    model = LSTMWithAttention(input_dim=len(lstm_features_cols))
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    stopper = EarlyStopping(patience=15)

    for epoch in range(150):
        model.train()
        for X_b, y_b in train_loader:
            optimizer.zero_grad()
            loss = ((model(X_b) - y_b) ** 2).mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        model.eval()
        v_losses = []
        with torch.no_grad():
            for X_b, y_b in val_loader:
                v_losses.append(((model(X_b) - y_b) ** 2).mean().item())
        if stopper.step(np.mean(v_losses), model):
            break

    stopper.restore_best(model)

    model.eval()
    preds, actuals = [], []
    with torch.no_grad():
        for X_b, y_b in test_loader:
            preds.append(model(X_b).numpy())
            actuals.append(y_b.numpy())

    attn_results[ticker] = {
        'pred': np.concatenate(preds),
        'actual': np.concatenate(actuals),
        'model': model,
        'test_loader': test_loader
    }

print(f'Attention LSTM trained for {len(attn_results)} stocks')

In [ ]:
# --- Compare vanilla LSTM vs attention LSTM ---
comparison_rows = []
for ticker in sample_tickers:
    n_t = min(len(lstm_results[ticker]['pred']), len(attn_results[ticker]['pred']))
    actual = lstm_results[ticker]['actual'][:n_t]
    lstm_p = lstm_results[ticker]['pred'][:n_t]
    attn_p = attn_results[ticker]['pred'][:n_t]

    comparison_rows.append({
        'Ticker': ticker,
        'LSTM MSE': compute_mse(actual, lstm_p),
        'Attn MSE': compute_mse(actual, attn_p),
        'LSTM QLIKE': compute_qlike(actual, lstm_p),
        'Attn QLIKE': compute_qlike(actual, attn_p),
        'LSTM Corr': compute_corr(actual, lstm_p),
        'Attn Corr': compute_corr(actual, attn_p)
    })

comp_df = pd.DataFrame(comparison_rows).set_index('Ticker')
print('Vanilla LSTM vs Attention LSTM:')
print(comp_df.round(6).to_string())

print(f'\nAverage improvement in QLIKE: '
      f'{(comp_df["LSTM QLIKE"] - comp_df["Attn QLIKE"]).mean():.6f}')
print(f'Average improvement in Correlation: '
      f'{(comp_df["Attn Corr"] - comp_df["LSTM Corr"]).mean():.4f}')

In [ ]:
# --- Visualize attention weights ---
# Show attention for a few test sequences from the first stock

ticker = sample_tickers[0]
model = attn_results[ticker]['model']
test_loader = attn_results[ticker]['test_loader']

# Get first batch
X_sample, y_sample = next(iter(test_loader))
weights = model.get_attention_weights(X_sample[:5])  # first 5 sequences

fig, axes = plt.subplots(5, 1, figsize=(12, 10), sharex=True)
for i in range(5):
    axes[i].bar(range(SEQ_LEN), weights[i].numpy(), color='steelblue')
    axes[i].set_ylabel(f'Sample {i+1}')
    axes[i].set_ylim(0, weights[i].max().item() * 1.3)

axes[-1].set_xlabel('Time Step (days before prediction)')
plt.suptitle(f'Attention Weights ({ticker})', fontsize=14)
plt.tight_layout()
plt.show()

# Average attention profile
all_weights = []
for X_b, _ in test_loader:
    w = model.get_attention_weights(X_b)
    all_weights.append(w.numpy())
avg_weights = np.concatenate(all_weights).mean(axis=0)

plt.figure(figsize=(10, 4))
plt.bar(range(SEQ_LEN), avg_weights, color='steelblue')
plt.xlabel('Time Step (days before prediction)')
plt.ylabel('Average Attention Weight')
plt.title(f'Average Attention Profile ({ticker})')
plt.tight_layout()
plt.show()

### Discussion

**1. Does attention help?**

The attention mechanism provides a modest improvement for some stocks. The benefit is most apparent for stocks with strong regime-dependent dynamics, where different historical time steps carry different amounts of information. For stocks with simple GARCH-like dynamics, the vanilla LSTM (which relies on the final hidden state) already captures the signal well, and attention adds little.

**2. What do the attention weights focus on?**

The average attention profile shows that the model tends to put more weight on recent time steps (especially the last 3-5 days), which is consistent with the strong short-term autocorrelation of realized volatility. However, there is also non-trivial weight on more distant time steps, particularly around the weekly lag (5 days back), suggesting the model has learned something akin to the HAR structure.

**3. Is the improvement worth the complexity?**

For this dataset, the improvement is marginal. The attention mechanism adds interpretability (we can see what the model focuses on), but the practical forecasting gains are small. In production, the simplicity of the HAR model or vanilla LSTM might be preferred unless working with richer feature sets where attention can selectively combine different information sources.